In [ ]:
import warnings                                                                  
warnings.filterwarnings("ignore") 
import os, sys
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import pygrib
from metpy.units import units
import metpy.calc as mpcalc
import cmocean as cmo
import scipy.ndimage as ndimage
import urllib.request 
from colorama import Fore
import time
import matplotlib.colors as colors
import matplotlib.patches as mpatches
import matplotlib.patheffects as path_effects
from matplotlib.lines import Line2D
from matplotlib.offsetbox import AnchoredText

In [ ]:
#Choose the year/month/day/hour of your case
month = '11' #Leading zero **REQUIRED** for months 1-9
day = '01' #Leading zero **REQUIRED** for days 1-9
year = '1991' #1979-Present (data not availible for up to the most recent month)
hour = '18' #00, 06, 12, or 18 

#Selct your sector view (ie: what projection you want) ---> *Only Northern Hemisphere Projections Are Currently Availible
#NOTE: First time use of a projection will take time (1 to 5 minutes) for the code to run... be patient!
SECTOR = 'CONUS' #Options are 'CONUS', 'North America', 'Europe', or 'Asia'. (Europe has missing data at the prime meridian)

#Choose which synoptic maps you would like created - enter 'Y' for yes or 'N' for no

#250mb winds and geopotential heights
jet250 = 'Y' 

#500mb cyclonic vorticity, geopotential heights, and winds
vorticity500 = 'Y' 

#850mb theta-e and MSLP (useful for front identification)
thetaE850 = 'Y' 

#850mb winds and heights (useful for LLJ identification)
winds850 = 'Y' 

#925mb frontogenesis, geopotential heights, and winds
frontogenesis925 = 'Y' 

#Precipitable water and 925-400mb average wind speed
PWAT = 'Y' 

#Severe Weather Threat Index (or SWEAT) map. General indicator of potenital t-storm severity
SWEAT = 'Y'

#2m temp, freezing line, MSLP, and 1000-500mb thickness
surfaceTemp = 'Y'

#Sfc apparent temp
apparentTemp = 'Y'

#Composite of 250 winds, 500 vort/heights, 850 winds, MSLP, and PWAT
SynopticComposite = 'Y'

#MORE OPTIONS BELOW:

#Choose if you want your surface map 2m tempertures in °C or °F
TEMP = 'F' #Enter 'F' or 'C' 

#Choose the folder (within the folder you are currently in) that you want the maps to save to
saveLoc = "SavLoc/"

In [ ]:
#URL Fetching

#Quick string creation for URL fetching
date = year+month
date2 = year+month+day
date3 = year+month+day+hour

try:
    try:
        #Set the URL
        url = 'https://www.ncei.noaa.gov/oa/prod-cfs-reanalysis/6-hourly-by-pressure-level/'+year+'/'+date+'/'+date2+'/pgbh00.gdas.'+date3+'.grb2'

        #Set file name
        out_Name = 'CFSR_'+date2+'_'+hour+'z'

        #Download the file
        if os.path.isfile(out_Name) == False:
            urllib.request.urlretrieve(url, out_Name)

        #Set the model name for file name creation
        modelName = 'CFSR'

        #Set variaible for WindSlice selction
        windModelName = 'CFSR'
    except:
        #Set the URL
        url = 'https://www.ncei.noaa.gov/data/global-forecast-system/access/grid-004-0.5-degree/analysis/'+date+'/'+date2+'/gfs_4_'+date2+'_'+hour+'00_000.grb2'

        #Set file name
        out_Name = 'GFS_'+date2+'_'+hour+'z'

        #Download the file
        if os.path.isfile(out_Name) == False:
            urllib.request.urlretrieve(url, out_Name)

        #Set the model name for file name creation
        modelName = 'GFS'

        #Set variaible for WindSlice selction
        if int(year) < 2021 or (int(year) == 2021 and int(month) < 7) or (int(year) == 2021 and int(month) == 7 and int(day) < 28):
            windModelName = 'GFS-old'
        else:
            windModelName = 'GFS-new'
except:
        #Set the URL
        url = 'https://www.ncei.noaa.gov/data/global-forecast-system/access/historical/analysis/'+date+'/'+date2+'/gfsanl_3_'+date2+'_'+hour+'00_000.grb'

        #Set file name
        out_Name = 'GFS_'+date2+'_'+hour+'z'

        #Download the file
        urllib.request.urlretrieve(url, out_Name)

        #Set the model name for file name creation
        modelName = 'GFS'

        #Set variaible for WindSlice selction
        windModelName = 'GFS-old'

In [ ]:
#This opens the grb file and displays all varibles present in the file
grbs = pygrib.open(out_Name)
grbs.seek(0)

for grb in grbs:
    print(grb)

In [ ]:
#Find Time & Date Variables in grb File 
T = grb.validDate

T = str(T)    

times = T[11:13]

times = "%sz" % (times)

year = T[0:4]

month = T[5:7]

day = T[8:10]

#Create strings for file names
date = "%s/%s/%s" % (month, day, year)
saveDate = "%s-%s-%s" % (month, day, year)

#Create text string for map title
validT = "Valid: %s %s" % (times, date)

#Create File Names For Later
mapNameJet250 = "250mbJet_"+modelName
fileNameJet250 = "%s%s_%s_%s.png" % (saveLoc, mapNameJet250, saveDate, times)

mapNameVorticity500 = "500mbVorticity_"+modelName
fileNameVorticity500 = "%s%s_%s_%s.png" % (saveLoc, mapNameVorticity500, saveDate, times)

mapNameThetaE850 = "850mbThetaE_"+modelName
fileNameThetaE850 = "%s%s_%s_%s.png" % (saveLoc, mapNameThetaE850, saveDate, times)

mapNameLLJ850 = "850mbWinds_"+modelName
fileNameLLJ850 = "%s%s_%s_%s.png" % (saveLoc, mapNameLLJ850, saveDate, times)

mapNameFrontogenesis925 = "925mbFrontogenesis_"+modelName
fileNameFrontogenesis925 = "%s%s_%s_%s.png" % (saveLoc, mapNameFrontogenesis925, saveDate, times)

mapNamePWAT = "PWAT_"+modelName
fileNamePWAT = "%s%s_%s_%s.png" % (saveLoc, mapNamePWAT, saveDate, times)

mapNameCAPE = "CAPE_"+modelName
fileNameCAPE = "%s%s_%s_%s.png" % (saveLoc, mapNameCAPE, saveDate, times)

mapNameSWEAT = "SWEAT_"+modelName
fileNameSWEAT = "%s%s_%s_%s.png" % (saveLoc, mapNameSWEAT, saveDate, times)

mapNameSurfaceTemp = "SurfaceTemp_"+modelName
fileNameSurfaceTemp = "%s%s_%s_%s.png" % (saveLoc, mapNameSurfaceTemp, saveDate, times)

mapNameApparentTemp = "ApparentTemp_"+modelName
fileNameApparentTemp = "%s%s_%s_%s.png" % (saveLoc, mapNameApparentTemp, saveDate, times)

mapNameSynopticComposite = "SynopticComposite_"+modelName
fileNameSynopticComposite = "%s%s_%s_%s.png" % (saveLoc, mapNameSynopticComposite, saveDate, times)

In [ ]:
# Handling for setting the sector view:
if SECTOR == 'CONUS':
    datacrs = ccrs.PlateCarree()
    EXTENT = [-127.5, -64.5, 20, 55]
    if windModelName == 'GFS-old':
        WINDSLICE = 3
    else:
        WINDSLICE = 6
elif SECTOR == 'North America':    
    datacrs = ccrs.LambertConformal()
    EXTENT = [-148, -48, 8, 74]
    if windModelName == 'GFS-old':
        WINDSLICE = 5
    else:
        WINDSLICE = 11
elif SECTOR == 'Europe':    
    datacrs = ccrs.Robinson()
    EXTENT = [-20, 40.5, 35, 72]
    if windModelName == 'GFS-old':
        WINDSLICE = 3
    else:
        WINDSLICE = 6
elif SECTOR == 'Asia':    
    datacrs = ccrs.PlateCarree()
    EXTENT = [27, 180, 5, 78.2]
    if windModelName == 'GFS-old':
        WINDSLICE = 6
    else:
        WINDSLICE = 12
    
#Set script created by text:
UpperRight = "Script By: @KniprathWx"

#You caught me! In code handling for knots still exists, but will potentially break maps, so change at your own risk...
#Choose if you want wind speed in mph or knots (this applies to ALL maps)
WIND = 'kt' #Enter 'mph' or 'kt'

#Code for High/Low Plotting Bypass
if (surfaceTemp == 'Y') or (SynopticComposite == 'Y'):
    ax54 = plt.subplot(1, 1, 1, projection=datacrs)
    ax54.set_extent(EXTENT, datacrs)
    print("Ignore this white box... It's a simple work around for High/Low Plotting!")
else:
    pass

In [ ]:
def JetMap250():
    #Reset the grb file
    grbs.seek(0)
    grb = grbs[1]

    #Data selection for 250mb Heights
    HGT_250mb = grbs.select(name='Geopotential height', typeOfLevel='isobaricInhPa', level=250)[0]
    HGT_250mb = HGT_250mb.values
    HGT_250mb = units('meter') * HGT_250mb

    #Data selection for U Wind Component at 250mb
    UWND_250mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=250)[0]
    UWND_250mb = UWND_250mb.values
    UWND_250mb = units('m/s') * UWND_250mb

    #Data selection for V Wind Component at 250mb
    VWND_250mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=250)[0]
    VWND_250mb = VWND_250mb.values
    VWND_250mb = units('m/s') * VWND_250mb

    #Calculate wind speed at 250mb
    sped_250mb = mpcalc.wind_speed(UWND_250mb, VWND_250mb).to(WIND)

    #Convert units to decameters for geopotential heights
    HGT_250mb = HGT_250mb / 10

    #Apply slight smoothing to parameters
    sped_250mb = ndimage.gaussian_filter(sped_250mb, sigma=1, order=0)
    HGT_250mb = ndimage.gaussian_filter(HGT_250mb, sigma=1, order=0)

    #Take in lats and lons from grb file
    lats, lons = grb.latlons()

    #Creating the map figure
    map_crs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
    fig = plt.figure(1, figsize=(14, 12), facecolor='black')
    ax = plt.subplot(1, 1, 1, projection=datacrs )
    ax.set_extent(EXTENT, ccrs.PlateCarree())

    #Add features to the map
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '10m', facecolor='steelblue', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', facecolor='gainsboro', zorder=52))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', facecolor='steelblue', linewidth=0.5, zorder=60))    
    ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '10m', facecolor='none', edgecolor='black', linewidth=0.5, zorder=70))
    if SECTOR == 'CONUS' or SECTOR == 'North America':
        ax.add_feature(cfeature.STATES, edgecolor='black', linewidth=0.5, zorder=70)
    else:
        pass

    #Setting up the ranges for contoured wind speeds
    #clevs_sped_250mb = [50, 75, 100, 125, 150, 175, 200, 225, 250]
    clevs_sped_250mb = [50, 75, 100, 120, 140, 160, 180, 200, 220]

    #*OLD* Custom Colortable
    #Creating a custom colormap with matplotlib colors 
    #cmap_data = ['yellow', 'orange', 'orangered', 'saddlebrown', 'darkviolet', 'deeppink', 'violet', 'lavenderblush']
    #cmapWIND = colors.ListedColormap(cmap_data, 'temperature')
    #norm = colors.BoundaryNorm(clevs_sped_250mb, cmapWIND.N)

    #Small function to truncate the wind speed color map to make it more visulally appealing
    def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
        new_cmap = colors.LinearSegmentedColormap.from_list(
            'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
            cmap(np.linspace(minval, maxval, n)))
        return new_cmap

    #arr = np.linspace(0, 50, 100).reshape((10, 10))
    #fig, ax = plt.subplots(ncols=2)

    cmap = plt.get_cmap('gist_ncar')
    gist_ncar_REFORMED = truncate_colormap(cmap, 0.18, 1)
    #ax[0].imshow(arr, interpolation='nearest', cmap=cmap)  
    #ax[1].imshow(arr, interpolation='nearest', cmap=gist_ncar_REFORMED)
    #plt.show()

    #Plot colorfill for wind speeds at 250mb
    cf = ax.contourf(lons, lats, sped_250mb, clevs_sped_250mb, cmap=cmo.tools.lighten(gist_ncar_REFORMED, 0.6), transform=ccrs.PlateCarree(), zorder=65)

    #Plot a grey contour line at transitions values for wind speeds at 250mb
    cs2 = ax.contour(lons, lats, sped_250mb, clevs_sped_250mb, colors='grey', transform=ccrs.PlateCarree(), alpha=0.6, linewidths=1, zorder=66)

    #Add colorbar for shaded wind speeds at 250mb
    cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50)
    cb.set_label('Wind Speed ('+WIND+')', color='white')
    cb.ax.xaxis.set_tick_params(color='white')
    plt.setp(plt.getp(cb.ax.axes, 'xticklabels'), color='white')
    cb.outline.set_edgecolor('white')

    #Plot contours for geopotential heights at 250mb
    clevs_HGT_250mb = np.arange(0, 1200, 12)
    cs = ax.contour(lons, lats, HGT_250mb, clevs_HGT_250mb, colors='black', transform=ccrs.PlateCarree(), zorder=80)
    plt.clabel(cs, fmt='%d')

    #Plot wind barbs every sixth element
    wind_slice = (slice(None, None, WINDSLICE), slice(None, None, WINDSLICE))           
    ax.barbs(lons[wind_slice], lats[wind_slice], UWND_250mb.to(WIND)[wind_slice].m, VWND_250mb[wind_slice].to(WIND).m, pivot='middle', color='black', transform=ccrs.PlateCarree(), length=6.5, zorder=67)

    #Add titles
    plt.title('250mb '+modelName+' Geopotential Heights (black, dam), Wind Speed (shaded, '+WIND+'), and Wind Barbs ('+WIND+')', loc='left', fontweight="bold", fontsize= 11, color='white')
    plt.title(validT, loc='right', fontsize= 8, color='white')

    UpperRightTitle = AnchoredText(UpperRight, loc=1, prop=dict(color='white', fontweight="bold", fontsize=8, horizontalalignment='right'), zorder=100)
    UpperRightTitle.patch.set_fc('black')
    UpperRightTitle.patch.set_alpha(0.7)
    ax.add_artist(UpperRightTitle)

    #Save figure to your computer (save loction outlined near date selection)
    plt.savefig(fileNameJet250, dpi=200)

    #Clear figure variables for next figure creation
    plt.clf()

In [ ]:
def VorticityMap500():
    #Reset the grb file
    grbs.seek(0)
    grb = grbs[1]

    #Data selection for 500mb Heights
    HGT_500mb = grbs.select(name='Geopotential height', typeOfLevel='isobaricInhPa', level=500)[0]
    HGT_500mb = HGT_500mb.values
    HGT_500mb = units('meter') * HGT_500mb

    #Data selection for U Wind Component at 500mb
    UWND_500mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=500)[0]
    UWND_500mb = UWND_500mb.values
    UWND_500mb = units('m/s') * UWND_500mb

    #Data selection for V Wind Component at 500mb
    VWND_500mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=500)[0]
    VWND_500mb = VWND_500mb.values
    VWND_500mb = units('m/s') * VWND_500mb

    #Data selection for Vorticity at 500mb
    VORT_500mb = grbs.select(name='Absolute vorticity', typeOfLevel='isobaricInhPa', level=500)[0]
    VORT_500mb = VORT_500mb.values
    VORT_500mb = units('1/s') * VORT_500mb

    #Convert units to decameters for geopotential heights
    HGT_500mb = HGT_500mb / 10

    #Convert 500mb Vorticity into usable units
    VORT_500mb = VORT_500mb * (1 / 10 ** -5)

    #Apply slight smoothing to parameters
    VORT_500mb = ndimage.gaussian_filter(VORT_500mb, sigma=1, order=0)
    HGT_500mb = ndimage.gaussian_filter(HGT_500mb, sigma=1, order=0)

    #Small function to truncate the vorticity color map to make it more visulally appealing
    def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
        new_cmap = colors.LinearSegmentedColormap.from_list(
            'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
            cmap(np.linspace(minval, maxval, n)))
        return new_cmap

    #arr = np.linspace(0, 50, 100).reshape((10, 10))
    #fig, ax = plt.subplots(ncols=2)

    cmap = plt.get_cmap('hot_r')
    HOT_R = truncate_colormap(cmap, 0.10, 1)
    #ax[0].imshow(arr, interpolation='nearest', cmap=cmap)  
    #ax[1].imshow(arr, interpolation='nearest', cmap=new_cmap)
    #plt.show()

    #Take in lats and lons from grb file
    lats, lons = grb.latlons()

    #Creating the map figure
    map_crs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
    fig = plt.figure(1, figsize=(14, 12), facecolor='black')
    ax = plt.subplot(1, 1, 1, projection=datacrs )
    ax.set_extent(EXTENT, ccrs.PlateCarree())

    #Add features to the map
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '10m', facecolor='steelblue', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', facecolor='gainsboro', zorder=52))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', facecolor='steelblue', linewidth=0.5, zorder=60))    
    ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '10m', facecolor='none', edgecolor='black', linewidth=0.5, zorder=70))
    if SECTOR == 'CONUS' or SECTOR == 'North America':
        ax.add_feature(cfeature.STATES, edgecolor='black', linewidth=0.5, zorder=70)
    else:
        pass

    #Add features to the map
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '10m', facecolor='steelblue', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', facecolor='gainsboro', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', facecolor='steelblue', zorder=60))
    ax.add_feature(cfeature.STATES, edgecolor='black', linewidth=1, zorder=70)    
    ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '10m', facecolor='none', edgecolor='black', linewidth=0.5, zorder=70))
    #Plot colorfill of 500mb vorticity
    clevs_VORT = np.arange(10,51,1)
    cf = ax.contourf(lons, lats, VORT_500mb, clevs_VORT, cmap=HOT_R, transform=ccrs.PlateCarree(), zorder=65)

    #Add colorbar for shaded vorticity at 500mb
    cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50)
    cb.set_label('Cyclonic Vorticity (1/sec*10^-5)', color='white')
    cb.ax.xaxis.set_tick_params(color='white')
    plt.setp(plt.getp(cb.ax.axes, 'xticklabels'), color='white')
    cb.outline.set_edgecolor('white')

    #Plot contours for geopotential heights at 500mb
    clevs_HGT_500mb = np.arange(0, 800, 6)
    cs = ax.contour(lons, lats, HGT_500mb, clevs_HGT_500mb, colors='black', transform=ccrs.PlateCarree(), zorder=80)
    plt.clabel(cs, fmt='%d')

    #Plot wind barbs every sixth element
    wind_slice = (slice(None, None, WINDSLICE), slice(None, None, WINDSLICE)) 
    ax.barbs(lons[wind_slice], lats[wind_slice], UWND_500mb.to(WIND)[wind_slice].m, VWND_500mb[wind_slice].to(WIND).m, pivot='middle', color='black', transform=ccrs.PlateCarree(), length=6.5, zorder=67)

    #Add titles
    plt.title('500mb '+modelName+' Geo. Heights (black, dam), Cyclonic Vorticity (shaded, 1/sec*10^-5), and Wind Barbs ('+WIND+')', loc='left', fontweight="bold", fontsize= 11, color='white')
    plt.title(validT, loc='right', fontsize= 8, color='white')

    UpperRightTitle = AnchoredText(UpperRight, loc=1, prop=dict(color='white', fontweight="bold", fontsize=8, horizontalalignment='right'), zorder=100)
    UpperRightTitle.patch.set_fc('black')
    UpperRightTitle.patch.set_alpha(0.7)
    ax.add_artist(UpperRightTitle)

    #Save figure to your computer (save loction outlined near date selection)
    plt.savefig(fileNameVorticity500, dpi=200)

    #Clear figure variables for next figure creation
    plt.clf()

In [ ]:
def ThetaE850():
    #Reset the grb file
    grbs.seek(0)
    grb = grbs[1]

    #Data selection for temperature at 850mb
    TMP_850mb = grbs.select(name='Temperature', typeOfLevel='isobaricInhPa', level=850)[0]
    TMP_850mb = TMP_850mb.values
    TMP_850mb = units('kelvin') * TMP_850mb

    #Data selection for RH at 850mb
    RH_850mb = grbs.select(name='Relative humidity', typeOfLevel='isobaricInhPa', level=850)[0]
    RH_850mb = RH_850mb.values
    RH_850mb = units('%') * RH_850mb

        #Data selection for MSLP
    try: #This is handling for CFS vs GFS data as varible names differ
        MSLP = grbs.select(name='MSLP (Eta model reduction)')[0]
    except:
        MSLP = grbs.select(name='Mean sea level pressure')[0] 
    MSLP = MSLP.values
    MSLP = units('Pa') * MSLP
    MSLP = MSLP.to('hPa')

    #Calculate dew point at 850mb
    DEW_850mb = mpcalc.dewpoint_from_relative_humidity(TMP_850mb, RH_850mb)

    #Calculate theta-e at 850mb
    THETAE_850mb = mpcalc.equivalent_potential_temperature(850 * units.mbar, TMP_850mb, DEW_850mb)

    #Apply slight smoothing to parameters
    THETAE_850mb = ndimage.gaussian_filter(THETAE_850mb, sigma=1, order=0)
    MSLP = ndimage.gaussian_filter(MSLP, sigma=1, order=0)

    #Funtion for plotting Highs and Lows
    def plot_maxmin_points(lon, lat, data, extrema, nsize, symbol, color='k', plotValue=True, transform=ccrs.PlateCarree()):

        from scipy.ndimage import maximum_filter, minimum_filter

        if (extrema == 'max'):
            data_ext = maximum_filter(data, nsize, mode='nearest')
        elif (extrema == 'min'):
            data_ext = minimum_filter(data, nsize, mode='nearest')
        else:
            raise ValueError('Value for hilo must be either max or min')

        mxy, mxx = np.where(data_ext == data)

        xmin, xmax, ymin, ymax = ax54.get_extent()
        yoffset = 0.022*(ymax-ymin)

        for i in range(len(mxy)):
            if (lon[mxy[i], mxx[i]] < ((xmax % 360)-yoffset)).all() and (lon[mxy[i], mxx[i]] > ((xmin % 360)+yoffset)).all() and (lat[mxy[i], mxx[i]] < (ymax-yoffset)).all() and (lat[mxy[i], mxx[i]] > (ymin+yoffset)).all():
                HLSymbol = ax.text(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]], symbol, color=color, size=24, clip_on=True, horizontalalignment='center', verticalalignment='center', transform=transform, zorder=95)
                HLSymbol.set_path_effects([path_effects.Stroke(linewidth=1.5, foreground='black'), path_effects.SimpleLineShadow(),path_effects.Normal()])
                HLvalue = ax.text(lon[mxy[i], mxx[i]], (lat[mxy[i], mxx[i]]-yoffset), str(int(data[mxy[i], mxx[i]])), color=color, size=10, clip_on=True, fontweight='bold', horizontalalignment='center', verticalalignment='top', transform=transform, zorder=95)                
                HLvalue.set_path_effects([path_effects.Stroke(linewidth=1, foreground='black'), path_effects.SimpleLineShadow(),path_effects.Normal()])

    #Take in lats and lons from grb file
    lats, lons = grb.latlons()

    #Creating the map figure
    map_crs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
    fig = plt.figure(1, figsize=(14, 12), facecolor='black')
    ax = plt.subplot(1, 1, 1, projection=datacrs )
    ax.set_extent(EXTENT, ccrs.PlateCarree())

    #Add features to the map
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '10m', facecolor='steelblue', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', facecolor='gainsboro', zorder=52))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', facecolor='steelblue', linewidth=0.5, zorder=60))    
    ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '10m', facecolor='none', edgecolor='black', linewidth=0.5, zorder=70))
    if SECTOR == 'CONUS' or SECTOR == 'North America':
        ax.add_feature(cfeature.STATES, edgecolor='black', linewidth=0.5, zorder=70)
    else:
        pass

    #Setting up the ranges for contoured theta-e
    clevs_THETAE = np.arange(240, 381, 2)

    #Creating a custom colortable using RGB values
    cmap_data = [(50/255,50/255,50/255),(73/255,73/255,73/255),(97/255,97/255,97/255),(121/255,121/255,121/255),(145/255,145/255,145/255),(182/255,168/255,186/255),(178/255,153/255,199/255),(187/255,141/255,227/255),(170/255,101/255,230/255),(154/255,67/255,230/255),(112/255,43/255,171/255),(81/255,29/255,125/255),(58/255,23/255,87/255),(23/255,23/255,87/255),(49/255,49/255,97/255),(49/255,49/255,121/255),(49/255,49/255,137/255),(49/255,49/255,153/255),(49/255,49/255,169/255),(49/255,49/255,193/255),(49/255,49/255,220/255),(49/255,49/255,251/255),(74/255,74/255,253/255),(81/255,83/255,246/255),(90/255,92/255,246/255),(110/255,112/255,245/255),(130/255,132/255,246/255),(151/255,153/255,245/255),(171/255,173/255,245/255),(192/255,194/255,245/255),(212/255,214/255,246/255),(233/255,235/255,246/255),(222/255,240/255,224/255),(182/255,229/255,184/255),(141/255,220/255,143/255),(59/255,199/255,61/255),(141/255,220/255,42/255),(209/255,236/255,42/255),(243/255,237/255,42/255),(243/255,223/255,42/255),(253/255,212/255,49/255),(252/255,201/255,49/255),(252/255,187/255,49/255),(252/255,171/255,49/255),(252/255,157/255,49/255),(252/255,139/255,49/255),(252/255,122/255,49/255),(252/255,104/255,49/255), (252/255,85/255,49/255), (253/255,49/255,49/255), (234/255,42/255,42/255), (205/255,49/255,49/255),(172/255,42/255,42/255),(140/255, 42/255, 42/255),(110/255, 35/255, 35/255),(78/255, 35/255, 35/255),(78/255, 35/255, 61/255),(115/255, 52/255, 90/255),(156/255, 75/255, 123/255), (173/255, 71/255, 132/255),(214/255, 60/255, 152/255),(224/255, 119/255, 223/255),(237/255, 164/255, 236/255),(237/255, 197/255, 237/255),(214/255, 197/255, 214/255), (237/255, 225/255, 237/255),(245/255, 218/255, 218/255),(214/255, 169/255, 169/255),(150/255, 113/255, 113/255),(112/255, 82/255, 82/255)]
    cmap = colors.ListedColormap(cmap_data, 'temperature')
    norm = colors.BoundaryNorm(clevs_THETAE, cmap.N)

    #Plotting theta-e with colorfill
    cf = ax.contourf(lons, lats, THETAE_850mb, clevs_THETAE, cmap=cmo.tools.lighten(cmap, 0.7), transform=ccrs.PlateCarree(), zorder=65)

    #Creating a colorbar for theta-e
    cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, ticks=[240, 260, 280, 300, 320, 340, 360, 380])
    cb.set_label('Equivalent Potential Temperature (K)', color='white')
    cb.ax.xaxis.set_tick_params(color='white')
    plt.setp(plt.getp(cb.ax.axes, 'xticklabels'), color='white')
    cb.outline.set_edgecolor('white')

    #Plot a grey contour line at transition values for theta-e
    cs22 = ax.contour(lons, lats, THETAE_850mb, clevs_THETAE, linewidths=0.75, colors='grey', alpha=0.6, transform=ccrs.PlateCarree(), zorder=66)

    #Countoring MSLP
    cs3 = ax.contour(lons, lats, MSLP, range(900, 1050, 2), transform=ccrs.PlateCarree(), colors='k', linestyles='solid', zorder=77)
    cs3.clabel(fontsize=10, inline=1, inline_spacing=7, fmt='%i', rightside_up=True, use_clabeltext=True)
    plot_maxmin_points(lons, lats, MSLP, 'max', 20, symbol='H', color='b',  transform=ccrs.PlateCarree())
    plot_maxmin_points(lons, lats, MSLP, 'min', 20, symbol='L', color='r', transform=ccrs.PlateCarree())

    #Add titles
    plt.title('850mb '+modelName+' Theta-E (shaded, K), MSLP (black, hPa), and Highs/Lows', loc='left', fontweight="bold", fontsize= 11, color='white')
    plt.title(validT, loc='right', fontsize= 8, color='white')

    UpperRightTitle = AnchoredText(UpperRight, loc=1, prop=dict(color='white', fontweight="bold", fontsize=8, horizontalalignment='right'), zorder=100)
    UpperRightTitle.patch.set_fc('black')
    UpperRightTitle.patch.set_alpha(0.7)
    ax.add_artist(UpperRightTitle)

    #Save figure to your computer (save loction outlined near date selection)
    plt.savefig(fileNameThetaE850, dpi=200)

    #Clear figure variables for next figure creation
    plt.clf()

In [ ]:
def LLJMap850():
    #Reset the grb file
    grbs.seek(0)
    grb = grbs[1]

    #Data selection for 850mb Heights
    HGT_850mb = grbs.select(name='Geopotential height', typeOfLevel='isobaricInhPa', level=850)[0]
    HGT_850mb = HGT_850mb.values
    HGT_850mb = units('meter') * HGT_850mb

    #Data selection for U Wind Component at 250mb
    UWND_850mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=850)[0]
    UWND_850mb = UWND_850mb.values
    UWND_850mb = units('m/s') * UWND_850mb

    #Data selection for V Wind Component at 250mb
    VWND_850mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=850)[0]
    VWND_850mb = VWND_850mb.values
    VWND_850mb = units('m/s') * VWND_850mb

    #Calculate wind speed at 250mb
    sped_850mb = mpcalc.wind_speed(UWND_850mb, VWND_850mb).to(WIND)

    #Convert units to decameters for geopotential heights
    HGT_850mb = HGT_850mb / 10

    #Apply slight smoothing to parameters
    sped_850mb = ndimage.gaussian_filter(sped_850mb, sigma=1, order=0)
    HGT_850mb = ndimage.gaussian_filter(HGT_850mb, sigma=1, order=0)

    #Take in lats and lons from grb file
    lats, lons = grb.latlons()

    #Creating the map figure
    map_crs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
    fig = plt.figure(1, figsize=(14, 12), facecolor='black')
    ax = plt.subplot(1, 1, 1, projection=datacrs )
    ax.set_extent(EXTENT, ccrs.PlateCarree())

    #Add features to the map
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '10m', facecolor='steelblue', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', facecolor='gainsboro', zorder=52))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', facecolor='steelblue', linewidth=0.5, zorder=60))    
    ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '10m', facecolor='none', edgecolor='black', linewidth=0.5, zorder=70))
    if SECTOR == 'CONUS' or SECTOR == 'North America':
        ax.add_feature(cfeature.STATES, edgecolor='black', linewidth=0.5, zorder=70)
    else:
        pass

    #Setting up the ranges for contoured wind speeds
    clevs_sped_850mb = [20, 30, 40, 50, 60, 70, 80, 100, 120]

    #Creating a custom colormap with matplotlib colors 
    cmap_data = ['yellow', 'orange', 'orangered', 'saddlebrown', 'darkviolet', 'deeppink', 'violet', 'lavenderblush']
    cmapWIND = colors.ListedColormap(cmap_data, 'temperature')
    norm = colors.BoundaryNorm(clevs_sped_850mb, cmapWIND.N)

    #Plot colorfill for wind speeds at 850mb
    cf = ax.contourf(lons, lats, sped_850mb, clevs_sped_850mb, cmap=cmo.tools.lighten(cmapWIND, 0.6), transform=ccrs.PlateCarree(), zorder=65)

    #Plot a grey contour line at transitions values for wind speeds at 250mb
    cs2 = ax.contour(lons, lats, sped_850mb, clevs_sped_850mb, colors='grey', transform=ccrs.PlateCarree(), alpha=0.6, linewidths=1, zorder=66)

    #Add colorbar for shaded wind speeds at 850mb
    cb = plt.colorbar(cf, orientation='horizontal', spacing='proportional', pad=0, aspect=50)
    cb.set_label('Wind Speed ('+WIND+')', color='white')
    cb.ax.xaxis.set_tick_params(color='white')
    plt.setp(plt.getp(cb.ax.axes, 'xticklabels'), color='white')
    cb.outline.set_edgecolor('white')

    #Plot contours for geopotential heights at 850mb
    clevs_HGT_850mb = np.arange(0, 800, 5)
    cs = ax.contour(lons, lats, HGT_850mb, clevs_HGT_850mb, colors='black', transform=ccrs.PlateCarree(), zorder=80)
    plt.clabel(cs, fmt='%d')

    #Plot wind barbs every sixth element
    wind_slice = (slice(None, None, WINDSLICE), slice(None, None, WINDSLICE))           
    ax.barbs(lons[wind_slice], lats[wind_slice], UWND_850mb.to(WIND)[wind_slice].m, VWND_850mb[wind_slice].to(WIND).m, pivot='middle', color='black', transform=ccrs.PlateCarree(), length=6.5, zorder=67)

    #Add titles
    plt.title('850mb '+modelName+' Geopotential Heights (black, dam), Wind Speed (shaded, '+WIND+'), and Wind Barbs ('+WIND+')', loc='left', fontweight="bold", fontsize= 11, color='white')
    plt.title(validT, loc='right', fontsize= 8, color='white')

    UpperRightTitle = AnchoredText(UpperRight, loc=1, prop=dict(color='white', fontweight="bold", fontsize=8, horizontalalignment='right'), zorder=100)
    UpperRightTitle.patch.set_fc('black')
    UpperRightTitle.patch.set_alpha(0.7)
    ax.add_artist(UpperRightTitle)

    #Save figure to your computer (save loction outlined near date selection)
    plt.savefig(fileNameLLJ850, dpi=200)

    #Clear figure variables for next figure creation
    plt.clf()

In [ ]:
def FrontogenesisMap925():
    #Reset the grb file
    grbs.seek(0)
    grb = grbs[1]

    #Data selection for 925mb Heights
    HGT_925mb = grbs.select(name='Geopotential height', typeOfLevel='isobaricInhPa', level=925)[0]
    HGT_925mb = HGT_925mb.values
    HGT_925mb = units('meter') * HGT_925mb

    #Data selection for 925mb Temperatures
    TEMP_925mb = grbs.select(name='Temperature', typeOfLevel='isobaricInhPa', level=925)[0]
    TEMP_925mb = TEMP_925mb.values
    TEMP_925mb = units('kelvin') * TEMP_925mb

    #Data selection for U Wind Component at 925mb
    UWND_925mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=925)[0]
    UWND_925mb = UWND_925mb.values
    UWND_925mb = units('m/s') * UWND_925mb

    #Data selection for V Wind Component at 925mb
    VWND_925mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=925)[0]
    VWND_925mb = VWND_925mb.values
    VWND_925mb = units('m/s') * VWND_925mb

    #Setting variables for calculating frontogenesis
    level925 = 925 * units.hPa
    lats, lons = grb.latlons()
    dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)

    #Calculating theta at 925mb for frontogenesis calculation
    THETA_925mb = mpcalc.potential_temperature(level925, TEMP_925mb)

    #Calculating frontogenesis
    FRONTO_925mb = mpcalc.frontogenesis(THETA_925mb, UWND_925mb, VWND_925mb, dx, dy)
    convert_to_per_100km_3h = 1000*100*3600*3
    FRONTO_925mb = FRONTO_925mb * convert_to_per_100km_3h

    #Apply slight smoothing to parameters
    #FRONTO_925mb = ndimage.gaussian_filter(FRONTO_925mb, sigma=1, order=0) * units('K/hour')
    HGT_925mb = ndimage.gaussian_filter(HGT_925mb, sigma=1, order=0) * units.meter

    #Convert heights from m to dam
    HGT_925mb = HGT_925mb / 10

    #Take in lats and lons from grb file
    lats, lons = grb.latlons()

    #Creating the map figure
    map_crs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
    fig = plt.figure(1, figsize=(14, 12), facecolor='black')
    ax = plt.subplot(1, 1, 1, projection=datacrs )
    ax.set_extent(EXTENT, ccrs.PlateCarree())

    #Add features to the map
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '10m', facecolor='steelblue', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', facecolor='gainsboro', zorder=52))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', facecolor='steelblue', linewidth=0.5, zorder=60))    
    ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '10m', facecolor='none', edgecolor='black', linewidth=0.5, zorder=70))
    if SECTOR == 'CONUS' or SECTOR == 'North America':
        ax.add_feature(cfeature.STATES, edgecolor='black', linewidth=0.5, zorder=70)
    else:
        pass

    #Plot colorfill of 925mb Frontogenesis
    clevs_FRONTO = np.arange(0.5,15.5,0.5)
    cf = ax.contourf(lons, lats, FRONTO_925mb, clevs_FRONTO, cmap='cool', extend="max", transform=ccrs.PlateCarree(), zorder=65)

    #Add colorbar for shaded Frontogenesis at 925mb
    cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, ticks=[0.5, 3, 6, 9, 12, 15])
    cb.set_label('Frontogenesis (°C/100km/3hr)', color='white')
    cb.ax.xaxis.set_tick_params(color='white')
    plt.setp(plt.getp(cb.ax.axes, 'xticklabels'), color='white')
    cb.outline.set_edgecolor('white')

    #Plot contours for geopotential heights at 500mb
    clevs_HGT_925mb = np.arange(0, 300, 3)
    cs = ax.contour(lons, lats, HGT_925mb, clevs_HGT_925mb, colors='black', transform=ccrs.PlateCarree(), zorder=80)
    plt.clabel(cs, fmt='%d')

    #Plot wind barbs every sixth element
    wind_slice = (slice(None, None, WINDSLICE), slice(None, None, WINDSLICE)) 
    ax.barbs(lons[wind_slice], lats[wind_slice], UWND_925mb.to(WIND)[wind_slice].m, VWND_925mb[wind_slice].to(WIND).m, pivot='middle', color='black', transform=ccrs.PlateCarree(), length=6.5, zorder=67)

    #Add titles
    plt.title('925mb '+modelName+' Frontogenesis (shaded, °C/100km/3hr), Heights (black, dam), and Wind Barbs ('+WIND+')', loc='left', fontweight="bold", fontsize=11, color='white')
    plt.title(validT, loc='right', fontsize=8, color='white')

    UpperRightTitle = AnchoredText(UpperRight, loc=1, prop=dict(color='white', fontweight="bold", fontsize=8, horizontalalignment='right'), zorder=100)
    UpperRightTitle.patch.set_fc('black')
    UpperRightTitle.patch.set_alpha(0.7)
    ax.add_artist(UpperRightTitle)

    #Save figure to your computer (save loction outlined near date selection)
    plt.savefig(fileNameFrontogenesis925, dpi=200)

    #Clear figure variables for next figure creation
    plt.clf()

In [ ]:
def PWATmap():
    #Reset the grb file
    grbs.seek(0)
    grb = grbs[1]

    #Data selection for PWATs
    if modelName == 'GFS':
        PWAT = grbs.select(name='Precipitable water')[0]
    else:
        PWAT = grbs.select(name='Precipitable water', typeOfLevel="atmosphereSingleLayer")[0]
    PWAT = PWAT.values
    PWAT = units('mm') * PWAT
    PWAT = PWAT.to('inch')

    #Data selection for U Wind Component at 250mb
    UWND_925mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=925)[0]
    UWND_925mb = UWND_925mb.values
    UWND_925mb = units('m/s') * UWND_925mb

    #Data selection for V Wind Component at 250mb
    VWND_925mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=925)[0]
    VWND_925mb = VWND_925mb.values
    VWND_925mb = units('m/s') * VWND_925mb

    #Data selection for U Wind Component at 250mb
    UWND_850mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=850)[0]
    UWND_850mb = UWND_850mb.values
    UWND_850mb = units('m/s') * UWND_850mb

    #Data selection for V Wind Component at 250mb
    VWND_850mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=850)[0]
    VWND_850mb = VWND_850mb.values
    VWND_850mb = units('m/s') * VWND_850mb

    #Data selection for U Wind Component at 250mb
    UWND_700mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=700)[0]
    UWND_700mb = UWND_700mb.values
    UWND_700mb = units('m/s') * UWND_700mb

    #Data selection for V Wind Component at 250mb
    VWND_700mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=700)[0]
    VWND_700mb = VWND_700mb.values
    VWND_700mb = units('m/s') * VWND_700mb

    #Data selection for U Wind Component at 500mb
    UWND_600mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=600)[0]
    UWND_600mb = UWND_600mb.values
    UWND_600mb = units('m/s') * UWND_600mb

    #Data selection for V Wind Component at 500mb
    VWND_600mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=600)[0]
    VWND_600mb = VWND_600mb.values
    VWND_600mb = units('m/s') * VWND_600mb

    #Data selection for U Wind Component at 500mb
    UWND_500mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=500)[0]
    UWND_500mb = UWND_500mb.values
    UWND_500mb = units('m/s') * UWND_500mb

    #Data selection for V Wind Component at 500mb
    VWND_500mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=500)[0]
    VWND_500mb = VWND_500mb.values
    VWND_500mb = units('m/s') * VWND_500mb

    #Data selection for U Wind Component at 400mb
    UWND_400mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=400)[0]
    UWND_400mb = UWND_400mb.values
    UWND_400mb = units('m/s') * UWND_400mb

    #Data selection for V Wind Component at 400mb
    VWND_400mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=400)[0]
    VWND_400mb = VWND_400mb.values
    VWND_400mb = units('m/s') * VWND_400mb

    #Calculate average wind speed in the column
    UWND_AVG = (UWND_925mb + UWND_850mb + UWND_700mb + UWND_600mb + UWND_500mb + UWND_400mb) / 6
    VWND_AVG = (VWND_925mb + VWND_850mb + VWND_700mb + VWND_600mb + VWND_500mb + VWND_400mb) / 6

    #Calculate wind speed at 250mb
    sped_AVG = mpcalc.wind_speed(UWND_AVG, VWND_AVG).to(WIND)

    #Apply slight smoothing to parameters
    sped_AVG = ndimage.gaussian_filter(sped_AVG, sigma=1, order=0)

    #Apply slight smoothing to parameters
    PWAT = ndimage.gaussian_filter(PWAT, sigma=1, order=0)

    #Take in lats and lons from grb file
    lats, lons = grb.latlons()

    #Creating the map figure
    map_crs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
    fig = plt.figure(1, figsize=(14, 12), facecolor='black')
    ax = plt.subplot(1, 1, 1, projection=datacrs )
    ax.set_extent(EXTENT, ccrs.PlateCarree())

    #Add features to the map
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '10m', facecolor='steelblue', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', facecolor='gainsboro', zorder=52))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', facecolor='steelblue', linewidth=0.5, zorder=60))    
    ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '10m', facecolor='none', edgecolor='black', linewidth=0.5, zorder=70))
    if SECTOR == 'CONUS' or SECTOR == 'North America':
        ax.add_feature(cfeature.STATES, edgecolor='black', linewidth=0.5, zorder=70)
    else:
        pass

    #Plot colorfill of 925mb Frontogenesis
    clevs_PWAT = [0, 0.25, 0.50, 0.75, 1, 1.25, 1.5, 1.75, 2, 2.25, 2.5, 2.75, 3]

    #Creating a custom colormap with matplotlib colors 
    cmap_data = ['burlywood', 'cornsilk', 'palegreen', 'limegreen', 'darkgreen', 'mediumturquoise', 'teal', 'mediumpurple', 'yellow', 'coral', 'deeppink', 'purple']
    cmapPWAT = colors.ListedColormap(cmap_data, 'PWAT')
    norm = colors.BoundaryNorm(clevs_PWAT, cmapPWAT.N)

    cf = ax.contourf(lons, lats, PWAT, clevs_PWAT, cmap=cmo.tools.lighten(cmapPWAT, 0.7), transform=ccrs.PlateCarree(), zorder=65)

    #Add colorbar for shaded Frontogenesis at 925mb
    cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, spacing='proportional', ticks=[0, 0.25, 0.5, 0.75, 1, 1.5, 2, 2.5, 3])
    cb.set_label('Precipitable Water (inch)', color='white')
    cb.ax.xaxis.set_tick_params(color='white')
    plt.setp(plt.getp(cb.ax.axes, 'xticklabels'), color='white')
    cb.outline.set_edgecolor('white')

    #Plot a grey contour line at transition values for 2m temperatures
    cs22 = ax.contour(lons, lats, PWAT, clevs_PWAT, linewidths=0.70, colors='grey', linestyles='solid', alpha=0.6, transform=ccrs.PlateCarree(), zorder=66)

    #Plot wind barbs every sixth element
    wind_slice = (slice(None, None, WINDSLICE), slice(None, None, WINDSLICE)) 
    ax.barbs(lons[wind_slice], lats[wind_slice], UWND_AVG.to(WIND)[wind_slice].m, VWND_AVG[wind_slice].to(WIND).m, pivot='middle', color='black', transform=ccrs.PlateCarree(), length=6.5, zorder=67)

    #Add titles
    plt.title(modelName+' Precipitable Water (shaded, in) and 925-400mb Average Wind Speed (barbs, '+WIND+')', loc='left', fontweight="bold", fontsize= 11, color='white')
    plt.title(validT, loc='right', fontsize= 10, color='white')

    UpperRightTitle = AnchoredText(UpperRight, loc=1, prop=dict(color='white', fontweight="bold", fontsize=8, horizontalalignment='right'), zorder=100)
    UpperRightTitle.patch.set_fc('black')
    UpperRightTitle.patch.set_alpha(0.7)
    ax.add_artist(UpperRightTitle)

    #Save figure to your computer (save loction outlined near date selection)
    plt.savefig(fileNamePWAT, dpi=200)

    #Clear figure variables for next figure creation
    plt.clf()

In [ ]:
def SWEATmap():
    grbs.seek(0)
    grb = grbs[1]

    #Data selection for temperature at 850mb
    TMP_850mb = grbs.select(name='Temperature', typeOfLevel='isobaricInhPa', level=850)[0]
    TMP_850mb = TMP_850mb.values
    TMP_850mb = units('kelvin') * TMP_850mb

    #Data selection for RH at 850mb
    RH_850mb = grbs.select(name='Relative humidity', typeOfLevel='isobaricInhPa', level=850)[0]
    RH_850mb = RH_850mb.values
    RH_850mb = units('%') * RH_850mb

    #Calculate dew point at 850mb
    DEW_850mb = mpcalc.dewpoint_from_relative_humidity(TMP_850mb, RH_850mb)
    TMP_850mb = TMP_850mb.to('degC')

    TMP_500mb = grbs.select(name='Temperature', typeOfLevel='isobaricInhPa', level=500)[0]
    TMP_500mb = TMP_500mb.values
    TMP_500mb = units('kelvin') * TMP_500mb
    TMP_500mb = TMP_500mb.to('degC')

    #Data selection for U Wind Component at 250mb
    UWND_850mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=850)[0]
    UWND_850mb = UWND_850mb.values
    UWND_850mb = units('m/s') * UWND_850mb

    #Data selection for V Wind Component at 250mb
    VWND_850mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=850)[0]
    VWND_850mb = VWND_850mb.values
    VWND_850mb = units('m/s') * VWND_850mb

    #Calculate wind speed at 250mb
    sped_850mb = mpcalc.wind_speed(UWND_850mb, VWND_850mb).to('knot')
    direction_850mb = mpcalc.wind_direction(UWND_850mb, VWND_850mb)

    #Data selection for U Wind Component at 250mb
    UWND_500mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=500)[0]
    UWND_500mb = UWND_500mb.values
    UWND_500mb = units('m/s') * UWND_500mb

    #Data selection for V Wind Component at 250mb
    VWND_500mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=500)[0]
    VWND_500mb = VWND_500mb.values
    VWND_500mb = units('m/s') * VWND_500mb

    #Calculate wind speed at 250mb
    sped_500mb = mpcalc.wind_speed(UWND_500mb, VWND_500mb).to('knot')
    direction_500mb = mpcalc.wind_direction(UWND_500mb, VWND_500mb)

    Td850 = DEW_850mb.magnitude #850 DewPoint
    Td850[Td850 < 0] = 0
    T850 = TMP_850mb.magnitude #850 Temp
    T500 = TMP_500mb.magnitude #500 Temp
    TT = (T850 + Td850) - (2*T500) #TotalTotals Index
    f850 = sped_850mb.magnitude #850 Winds
    f500 = sped_500mb.magnitude #500 Winds
    dd850 = direction_850mb.magnitude #Wind Direction 8500
    dd500 = direction_500mb.magnitude #Wind Direction 500

    dd850[dd850 < 250] = 0
    dd850[dd850 > 130] = 0

    dd500[dd500 < 310] = 0
    dd500[dd500 > 210] = 0

    dd850[np.where(dd850-dd500 < 0)] = 0
    dd500[np.where(dd850-dd500 < 0)] = 0

    dd850[(dd850<15)&(dd500<15)] = 0
    dd500[(dd850<15)&(dd500<15)] = 0

    S = np.sin(dd850 - dd500)

    SWEAT = (12 * Td850) + (20*(TT-49)) + (2* f850) + f500 + (125*(S+0.2))

    SWEAT = ndimage.gaussian_filter(SWEAT, sigma=1, order=0)

    #Take in lats and lons from grb file
    lats, lons = grb.latlons()

    #Creating the map figure
    map_crs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
    fig = plt.figure(1, figsize=(14, 12), facecolor='black')
    ax = plt.subplot(1, 1, 1, projection=datacrs )
    ax.set_extent(EXTENT, ccrs.PlateCarree())

    #Add features to the map
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '10m', facecolor='steelblue', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', facecolor='gainsboro', zorder=52))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', facecolor='steelblue', linewidth=0.5, zorder=60))    
    ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '10m', facecolor='none', edgecolor='black', linewidth=0.5, zorder=70))
    if SECTOR == 'CONUS' or SECTOR == 'North America':
        ax.add_feature(cfeature.STATES, edgecolor='black', linewidth=0.5, zorder=70)
    else:
        pass

    # Plot SB_CAPE
    clevs_SWEAT = [150,200,250,300,350,400,450]
    cmap_SWEAT = colors.ListedColormap(['palegreen','palegreen','gold','indianred','indianred', 'blueviolet'])
    cf = ax.contourf(lons, lats, SWEAT, clevs_SWEAT, cmap=cmap_SWEAT, extend='max', transform=datacrs, zorder=65)

    patch200 = mpatches.Patch(color='palegreen', label='General')
    patch250 = mpatches.Patch(color='gold', label='Strong')
    patch300 = mpatches.Patch(color='indianred', label='Severe')
    patch400 = mpatches.Patch(color='blueviolet', label='Tornadoes')

    leg = plt.legend(handles=[patch200, patch250, patch300, patch400], loc=3, framealpha=1, fontsize="11")
    leg.set_zorder(100)

    #Add titles
    plt.title(modelName+' Severe Weather Threat (SWeaT) Index', loc='left', fontweight="bold", fontsize= 11, color='white')
    plt.title(validT, loc='right', fontsize= 10, color='white')

    BottomRightTitle = AnchoredText('This index should only be used as a preliminary identifier of atmospheric conditions potentially supportive of thunderstorm\n  severity. More extensive analysis should be completed to better understand overall severe weather potential and threats.', loc=4, prop=dict(color='white', fontweight="bold", fontsize=8, horizontalalignment='right'), zorder=100)
    BottomRightTitle.patch.set_fc('black')
    BottomRightTitle.patch.set_alpha(0.85)
    ax.add_artist(BottomRightTitle)

    UpperRightTitle = AnchoredText(UpperRight, loc=1, prop=dict(color='white', fontweight="bold", fontsize=8, horizontalalignment='right'), zorder=100)
    UpperRightTitle.patch.set_fc('black')
    UpperRightTitle.patch.set_alpha(0.7)
    ax.add_artist(UpperRightTitle)

    #Save figure to your computer (save loction outlined near date selection)
    plt.savefig(fileNameSWEAT, dpi=200)

    #Clear figure variables for next figure creation
    plt.clf()

In [ ]:
def SurfaceTempMap():
    #Reset the grb file
    grbs.seek(0)
    grb = grbs[1]

    #Data selection for MSLP
    try: #This is handling for CFS vs GFS data as varible names differ
        MSLP = grbs.select(name='MSLP (Eta model reduction)')[0]
    except:
        MSLP = grbs.select(name='Mean sea level pressure')[0] 
    MSLP = MSLP.values
    MSLP = units('Pa') * MSLP
    MSLP = MSLP.to('hPa')

    #Data selection for MSLP
    SFC_temp = grbs.select(name='2 metre temperature')[0]
    SFC_temp = SFC_temp.values
    SFC_temp = units('K') * SFC_temp
    if TEMP == 'F':
        SFC_temp = SFC_temp.to('degF')
    else:
        SFC_temp = SFC_temp.to('degC')

    #Apply slight smoothing to parameters
    MSLP = ndimage.gaussian_filter(MSLP, sigma=1, order=0)
    SFC_temp = ndimage.gaussian_filter(SFC_temp, sigma=1, order=0)

    #Take in lats and lons from grb file
    lats, lons = grb.latlons()

    #Creating the map figure
    map_crs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
    fig = plt.figure(1, figsize=(14, 12), facecolor='black')
    ax = plt.subplot(1, 1, 1, projection=datacrs )
    ax.set_extent(EXTENT, ccrs.PlateCarree())

    #Add features to the map
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '10m', facecolor='steelblue', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', facecolor='gainsboro', zorder=52))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', facecolor='steelblue', linewidth=0.5, zorder=60))    
    ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '10m', facecolor='none', edgecolor='black', linewidth=0.5, zorder=70))
    if SECTOR == 'CONUS' or SECTOR == 'North America':
        ax.add_feature(cfeature.STATES, edgecolor='black', linewidth=0.5, zorder=70)
    else:
        pass

    #If-else statement for temperture plotting °F vs °C
    if TEMP == 'F':
        #Setting up the ranges for contoured temperatures
        clevs = np.arange(-60, 121, 5)

        #Creating a custom colortable using RGB values
        cmap_data = [(205/255,18/255,86/255), (231/255,41/255,137/255), (222/255,101/255,176/255), (255/255,115/255,222/255), (249/255,191/255,229/255), (254/255,254/255,255/255), (219/255,218/255,234/255), (188/255,189/255,220/255), (157/255,154/255,199/255), (117/255,107/255,175/255), (84/255,39/255,142/255), (12/255,0/255,124/255), (13/255,64/255,159/255), (1/255,102/255,193/255), (41/255,158/255,254/255), (74/255,199/255,255/255), (114/255,215/255,255/255), (173/255,254/255,255/255), (49/255,207/255,193/255), (0/255,153/255,150/255), (18/255,87/255,87/255), (6/255,109/255,44/255), (49/255,163/255,84/255), (116/255,196/255,118/255), (161/255,217/255,155/255), (211/255,255/255,190/255), (255/255,255/255,179/255), (254/255,235/255,156/255), (255/255,219/255,120/255), (255/255,174/255,43/255), (253/255,140/255,60/255), (252/255,78/255,43/255), (226/255,26/255,28/255), (176/255,0/255,38/255), (129/255,0/255,38/255), (89/255,0/255,66/255)]
        cmap = colors.ListedColormap(cmap_data, 'temperature')
        norm = colors.BoundaryNorm(clevs, cmap.N)

        #Plotting 2m temperature with colorfill
        cf = ax.contourf(lons, lats, SFC_temp, np.arange(-60, 121, 5), cmap=cmap, extend="both", transform=ccrs.PlateCarree(), zorder=65)

        #Creating a colorbar for 2m temperature
        cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, ticks=[-60, -50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120])
        cb.set_label('2m Temperature (°F)', color='white')
        cb.ax.xaxis.set_tick_params(color='white')
        plt.setp(plt.getp(cb.ax.axes, 'xticklabels'), color='white')
        cb.outline.set_edgecolor('white')

        #Plot a grey contour line at transition values for 2m temperatures
        cs22 = ax.contour(lons, lats, SFC_temp, np.arange(-60, 121, 5), linewidths=0.75, colors='grey', linestyles='solid', alpha=0.6, transform=ccrs.PlateCarree(), zorder=66)

        #Plot the freezing line
        cs28 = ax.contour(lons, lats, SFC_temp, np.arange(32, 33, 1), linewidths=1.5, colors='white', alpha=1, linestyles='dashed', transform=ccrs.PlateCarree(), zorder=71)
    else:
        #Setting up the ranges for contoured temperatures
        clevs = np.arange(-51, 58, 3)

        #Creating a custom colortable using RGB values
        cmap_data = [(205/255,18/255,86/255), (231/255,41/255,137/255), (222/255,101/255,176/255), (255/255,115/255,222/255), (249/255,191/255,229/255), (254/255,254/255,255/255), (219/255,218/255,234/255), (188/255,189/255,220/255), (157/255,154/255,199/255), (117/255,107/255,175/255), (84/255,39/255,142/255), (12/255,0/255,124/255), (13/255,64/255,159/255), (1/255,102/255,193/255), (41/255,158/255,254/255), (74/255,199/255,255/255), (114/255,215/255,255/255), (173/255,254/255,255/255), (49/255,207/255,193/255), (0/255,153/255,150/255), (18/255,87/255,87/255), (6/255,109/255,44/255), (49/255,163/255,84/255), (116/255,196/255,118/255), (161/255,217/255,155/255), (211/255,255/255,190/255), (255/255,255/255,179/255), (254/255,235/255,156/255), (255/255,219/255,120/255), (255/255,174/255,43/255), (253/255,140/255,60/255), (252/255,78/255,43/255), (226/255,26/255,28/255), (176/255,0/255,38/255), (129/255,0/255,38/255), (89/255,0/255,66/255)]
        cmap = colors.ListedColormap(cmap_data, 'temperature')
        norm = colors.BoundaryNorm(clevs, cmap.N)

        #Plotting 2m temperature with colorfill
        cf = ax.contourf(lons, lats, SFC_temp, np.arange(-51, 58, 3), cmap=cmap, extend="both", transform=ccrs.PlateCarree(), zorder=65)

        #Creating a colorbar for 2m temperature
        cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, ticks=[-45,-30, -15, 0, 15, 30, 45])
        cb.set_label('2m Temperature (°C)', color='white')
        cb.ax.xaxis.set_tick_params(color='white')
        plt.setp(plt.getp(cb.ax.axes, 'xticklabels'), color='white')
        cb.outline.set_edgecolor('white')

        #Plot a grey contour line at transition values for 2m temperatures
        cs22 = ax.contour(lons, lats, SFC_temp, np.arange(-51, 58, 3), linewidths=0.75, colors='grey', linestyles='solid', alpha=0.6, transform=ccrs.PlateCarree(), zorder=66)

        #Plot the freezing line
        cs28 = ax.contour(lons, lats, SFC_temp, np.arange(0, 1, 1), linewidths=1.5, colors='white', alpha=1, linestyles='dashed', transform=ccrs.PlateCarree(), zorder=71)

    #Funtion for plotting Highs and Lows
    def plot_maxmin_points(lon, lat, data, extrema, nsize, symbol, color='k', plotValue=True, transform=ccrs.PlateCarree()):

        from scipy.ndimage import maximum_filter, minimum_filter

        if (extrema == 'max'):
            data_ext = maximum_filter(data, nsize, mode='nearest')
        elif (extrema == 'min'):
            data_ext = minimum_filter(data, nsize, mode='nearest')
        else:
            raise ValueError('Value for hilo must be either max or min')

        mxy, mxx = np.where(data_ext == data)

        xmin, xmax, ymin, ymax = ax54.get_extent()
        yoffset = 0.022*(ymax-ymin)

        for i in range(len(mxy)):
            if (lon[mxy[i], mxx[i]] < ((xmax % 360)-yoffset)).all() and (lon[mxy[i], mxx[i]] > ((xmin % 360)+yoffset)).all() and (lat[mxy[i], mxx[i]] < (ymax-yoffset)).all() and (lat[mxy[i], mxx[i]] > (ymin+yoffset)).all():
                HLSymbol = ax.text(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]], symbol, color=color, size=24, clip_on=True, horizontalalignment='center', verticalalignment='center', transform=transform, zorder=95)
                HLSymbol.set_path_effects([path_effects.Stroke(linewidth=2, foreground='black'), path_effects.SimpleLineShadow(),path_effects.Normal()])
                HLvalue = ax.text(lon[mxy[i], mxx[i]], (lat[mxy[i], mxx[i]]-yoffset), str(int(data[mxy[i], mxx[i]])), color=color, size=10, clip_on=True, fontweight='bold', horizontalalignment='center', verticalalignment='top', transform=transform, zorder=95)                
                HLvalue.set_path_effects([path_effects.Stroke(linewidth=1.5, foreground='black'), path_effects.SimpleLineShadow(),path_effects.Normal()])

    #Countoring MSLP and max/min points
    cs3 = ax.contour(lons, lats, MSLP, range(900, 1050, 4), transform=ccrs.PlateCarree(), colors='k', linestyles='solid', zorder=77)
    ax.clabel(cs3, inline=1, fontsize=9, fmt='%d')
    plot_maxmin_points(lons, lats, MSLP, 'max', 20, symbol='H', color='b',  transform=ccrs.PlateCarree())
    plot_maxmin_points(lons, lats, MSLP, 'min', 20, symbol='L', color='r', transform=ccrs.PlateCarree())

    #Variable haqndling for title   
    if TEMP == 'C':
        FLineTemp = "0°C"
    else:
        FLineTemp = "32°F"

    #Add titles
    plt.title(modelName+' 2m Temp (shaded, °'+TEMP+'), Freezing Line (dashed white, '+FLineTemp+'), MSLP (black, hPa), and Highs/Lows', loc='left', fontweight="bold", fontsize= 11, color='white')
    plt.title(validT, loc='right', fontsize= 10, color='white')

    UpperRightTitle = AnchoredText(UpperRight, loc=1, prop=dict(color='white', fontweight="bold", fontsize=8, horizontalalignment='right'), zorder=100)
    UpperRightTitle.patch.set_fc('black')
    UpperRightTitle.patch.set_alpha(0.7)
    ax.add_artist(UpperRightTitle)

    #Save figure to your computer (save loction outlined near date selection)
    plt.savefig(fileNameSurfaceTemp, dpi=200)

    #Clear figure variables for next figure creation
    plt.clf()

In [ ]:
def ApparentTempMap():
    #Reset the grb file
    grbs.seek(0)
    grb = grbs[1]

    #Data selection for MSLP
    SFC_temp = grbs.select(name='2 metre temperature')[0]
    SFC_temp = SFC_temp.values
    SFC_temp = units('K') * SFC_temp

    #Data selection for U Wind Component at the surface
    U_sfc = grbs.select(name='10 metre U wind component')[0]
    U_sfc = U_sfc.values
    U_sfc = units('m/s') * U_sfc

    #Data selection for V Wind Component at the surface
    V_sfc = grbs.select(name='10 metre V wind component')[0]
    V_sfc = V_sfc.values
    V_sfc = units('m/s') * V_sfc

    #Data selection for RH at the surface
    try:
        RH = grbs.select(name='Relative humidity', typeOfLevel='heightAboveGround', level=2)[0]
    except:
        RH = grbs.select(name='2 metre relative humidity')[0]
    RH = RH.values
    RH = units('%') * RH

    sfc_wind = mpcalc.wind_speed(U_sfc, V_sfc)

    if TEMP == 'F':
        apparent_temp = mpcalc.apparent_temperature(SFC_temp, RH, sfc_wind, face_level_winds=False, mask_undefined=True).to('degF')
    else:
        apparent_temp = mpcalc.apparent_temperature(SFC_temp, RH, sfc_wind, face_level_winds=False, mask_undefined=True).to('degC')

    #Apply slight smoothing to parameters
    apparent_temp = ndimage.gaussian_filter(apparent_temp, sigma=1, order=0)

    #Take in lats and lons from grb file
    lats, lons = grb.latlons()

    #Creating the map figure
    map_crs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
    fig = plt.figure(1, figsize=(14, 12), facecolor='black')
    ax = plt.subplot(1, 1, 1, projection=datacrs )
    ax.set_extent(EXTENT, ccrs.PlateCarree())

    #Add features to the map
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '10m', facecolor='steelblue', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', facecolor='gainsboro', zorder=52))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', facecolor='steelblue', linewidth=0.5, zorder=60))    
    ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '10m', facecolor='none', edgecolor='black', linewidth=0.5, zorder=70))
    if SECTOR == 'CONUS' or SECTOR == 'North America':
        ax.add_feature(cfeature.STATES, edgecolor='black', linewidth=0.5, zorder=70)
    else:
        pass

    #If-else statement for temperture plotting °F vs °C
    if TEMP == 'F':
        #Setting up the ranges for contoured temperatures
        clevs = np.arange(-60, 121, 5)

        #Creating a custom colortable using RGB values
        cmap_data = [(205/255,18/255,86/255), (231/255,41/255,137/255), (222/255,101/255,176/255), (255/255,115/255,222/255), (249/255,191/255,229/255), (254/255,254/255,255/255), (219/255,218/255,234/255), (188/255,189/255,220/255), (157/255,154/255,199/255), (117/255,107/255,175/255), (84/255,39/255,142/255), (12/255,0/255,124/255), (13/255,64/255,159/255), (1/255,102/255,193/255), (41/255,158/255,254/255), (74/255,199/255,255/255), (114/255,215/255,255/255), (173/255,254/255,255/255), (49/255,207/255,193/255), (0/255,153/255,150/255), (18/255,87/255,87/255), (6/255,109/255,44/255), (49/255,163/255,84/255), (116/255,196/255,118/255), (161/255,217/255,155/255), (211/255,255/255,190/255), (255/255,255/255,179/255), (254/255,235/255,156/255), (255/255,219/255,120/255), (255/255,174/255,43/255), (253/255,140/255,60/255), (252/255,78/255,43/255), (226/255,26/255,28/255), (176/255,0/255,38/255), (129/255,0/255,38/255), (89/255,0/255,66/255)]
        cmap = colors.ListedColormap(cmap_data, 'temperature')
        norm = colors.BoundaryNorm(clevs, cmap.N)

        #Plotting 2m temperature with colorfill
        cf = ax.contourf(lons, lats, apparent_temp, np.arange(-60, 121, 5), cmap=cmap, extend="both", transform=ccrs.PlateCarree(), zorder=65)

        #Creating a colorbar for 2m temperature
        cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, ticks=[-60, -50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120])
        cb.set_label('2m Apparent Temp (°F)', color='white')
        cb.ax.xaxis.set_tick_params(color='white')
        plt.setp(plt.getp(cb.ax.axes, 'xticklabels'), color='white')
        cb.outline.set_edgecolor('white')

        #Plot a grey contour line at transition values for 2m temperatures
        cs22 = ax.contour(lons, lats, apparent_temp, np.arange(-60, 121, 5), linewidths=0.75, colors='grey', linestyles='solid', alpha=0.6, transform=ccrs.PlateCarree(), zorder=66)

    else:
        #Setting up the ranges for contoured temperatures
        clevs = np.arange(-51, 58, 3)

        #Creating a custom colortable using RGB values
        cmap_data = [(205/255,18/255,86/255), (231/255,41/255,137/255), (222/255,101/255,176/255), (255/255,115/255,222/255), (249/255,191/255,229/255), (254/255,254/255,255/255), (219/255,218/255,234/255), (188/255,189/255,220/255), (157/255,154/255,199/255), (117/255,107/255,175/255), (84/255,39/255,142/255), (12/255,0/255,124/255), (13/255,64/255,159/255), (1/255,102/255,193/255), (41/255,158/255,254/255), (74/255,199/255,255/255), (114/255,215/255,255/255), (173/255,254/255,255/255), (49/255,207/255,193/255), (0/255,153/255,150/255), (18/255,87/255,87/255), (6/255,109/255,44/255), (49/255,163/255,84/255), (116/255,196/255,118/255), (161/255,217/255,155/255), (211/255,255/255,190/255), (255/255,255/255,179/255), (254/255,235/255,156/255), (255/255,219/255,120/255), (255/255,174/255,43/255), (253/255,140/255,60/255), (252/255,78/255,43/255), (226/255,26/255,28/255), (176/255,0/255,38/255), (129/255,0/255,38/255), (89/255,0/255,66/255)]
        cmap = colors.ListedColormap(cmap_data, 'temperature')
        norm = colors.BoundaryNorm(clevs, cmap.N)

        #Plotting 2m temperature with colorfill
        cf = ax.contourf(lons, lats, apparent_temp, np.arange(-51, 58, 3), cmap=cmap, extend="both", transform=ccrs.PlateCarree(), zorder=65)

        #Creating a colorbar for 2m temperature
        cb = plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, ticks=[-45,-30, -15, 0, 15, 30, 45])
        cb.set_label('2m Apparent Temp (°C)', color='white')
        cb.ax.xaxis.set_tick_params(color='white')
        plt.setp(plt.getp(cb.ax.axes, 'xticklabels'), color='white')
        cb.outline.set_edgecolor('white')

        #Plot a grey contour line at transition values for 2m temperatures
        cs22 = ax.contour(lons, lats, apparent_temp, np.arange(-51, 58, 3), linewidths=0.75, colors='grey', linestyles='solid', alpha=0.6, transform=ccrs.PlateCarree(), zorder=66)

    #Plot wind barbs every sixth element
    #wind_slice = (slice(None, None, WINDSLICE), slice(None, None, WINDSLICE)) 
    #ax.barbs(lons[wind_slice], lats[wind_slice], U_sfc.to(WIND)[wind_slice].m, V_sfc[wind_slice].to(WIND).m, pivot='middle', color='black', transform=ccrs.PlateCarree(), length=6.5, zorder=67)

    #Add titles
    plt.title(modelName+' Apparent Surface Temperature (shaded, °'+TEMP+')', loc='left', fontweight="bold", fontsize= 11, color='white')
    plt.title(validT, loc='right', fontsize= 10, color='white')

    UpperRightTitle = AnchoredText(UpperRight, loc=1, prop=dict(color='white', fontweight="bold", fontsize=8, horizontalalignment='right'), zorder=100)
    UpperRightTitle.patch.set_fc('black')
    UpperRightTitle.patch.set_alpha(0.7)
    ax.add_artist(UpperRightTitle)

    #Save figure to your computer (save loction outlined near date selection)
    plt.savefig(fileNameApparentTemp, dpi=200)

    #Clear figure variables for next figure creation
    plt.clf()

In [ ]:
 def SynopticCompositeMap():
    #Reset the grb file
    grbs.seek(0)
    grb = grbs[1]

    #Data selection for U Wind Component at 250mb
    UWND_250mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=250)[0]
    UWND_250mb = UWND_250mb.values
    UWND_250mb = units('m/s') * UWND_250mb

    #Data selection for V Wind Component at 250mb
    VWND_250mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=250)[0]
    VWND_250mb = VWND_250mb.values
    VWND_250mb = units('m/s') * VWND_250mb

    #Calculate wind speed at 250mb
    sped_250mb = mpcalc.wind_speed(UWND_250mb, VWND_250mb).to('m/s')

    #Data selection for 500mb Heights
    HGT_500mb = grbs.select(name='Geopotential height', typeOfLevel='isobaricInhPa', level=500)[0]
    HGT_500mb = HGT_500mb.values
    HGT_500mb = units('meter') * HGT_500mb

    #Data selection for Vorticity at 500mb
    VORT_500mb = grbs.select(name='Absolute vorticity', typeOfLevel='isobaricInhPa', level=500)[0]
    VORT_500mb = VORT_500mb.values
    VORT_500mb = units('1/s') * VORT_500mb

    #Data selection for U Wind Component at 250mb
    UWND_850mb = grbs.select(name='U component of wind', typeOfLevel='isobaricInhPa', level=850)[0]
    UWND_850mb = UWND_850mb.values
    UWND_850mb = units('m/s') * UWND_850mb

    #Data selection for V Wind Component at 250mb
    VWND_850mb = grbs.select(name='V component of wind', typeOfLevel='isobaricInhPa', level=850)[0]
    VWND_850mb = VWND_850mb.values
    VWND_850mb = units('m/s') * VWND_850mb

    #Data selection for PWATs
    if modelName == 'GFS':
        PWAT = grbs.select(name='Precipitable water')[0]
    else:
        PWAT = grbs.select(name='Precipitable water', typeOfLevel="atmosphereSingleLayer")[0]    
    PWAT = PWAT.values
    PWAT = units('mm') * PWAT
    PWAT = PWAT.to('inch')

    #Data selection for MSLP
    try: #This is handling for CFS vs GFS data as varible names differ
        MSLP = grbs.select(name='MSLP (Eta model reduction)')[0]
    except:
        MSLP = grbs.select(name='Mean sea level pressure')[0]

    MSLP = MSLP.values
    MSLP = units('Pa') * MSLP
    MSLP = MSLP.to('hPa')

    #Convert units to decameters for geopotential heights
    HGT_500mb = HGT_500mb / 10

    #Convert 500mb Vorticity into usable units
    VORT_500mb = VORT_500mb * 10**5

    #Apply slight smoothing to parameters
    sped_250mb = ndimage.gaussian_filter(sped_250mb, sigma=1, order=0)
    VORT_500mb = ndimage.gaussian_filter(VORT_500mb, sigma=1, order=0)
    HGT_500mb = ndimage.gaussian_filter(HGT_500mb, sigma=1, order=0)
    PWAT = ndimage.gaussian_filter(PWAT, sigma=1, order=0)
    MSLP = ndimage.gaussian_filter(MSLP, sigma=1, order=0)

    #Funtion for plotting Highs and Lows
    def plot_maxmin_points(lon, lat, data, extrema, nsize, symbol, color='k', plotValue=True, transform=ccrs.PlateCarree()):

        from scipy.ndimage import maximum_filter, minimum_filter

        if (extrema == 'max'):
            data_ext = maximum_filter(data, nsize, mode='nearest')
        elif (extrema == 'min'):
            data_ext = minimum_filter(data, nsize, mode='nearest')
        else:
            raise ValueError('Value for hilo must be either max or min')

        mxy, mxx = np.where(data_ext == data)

        xmin, xmax, ymin, ymax = ax54.get_extent()
        yoffset = 0.022*(ymax-ymin)

        for i in range(len(mxy)):
            if (lon[mxy[i], mxx[i]] < ((xmax % 360)-yoffset)).all() and (lon[mxy[i], mxx[i]] > ((xmin % 360)+yoffset)).all() and (lat[mxy[i], mxx[i]] < (ymax-yoffset)).all() and (lat[mxy[i], mxx[i]] > (ymin+yoffset)).all():
                HLSymbol = ax.text(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]], symbol, color=color, size=24, clip_on=True, horizontalalignment='center', verticalalignment='center', transform=transform, zorder=95)
                HLSymbol.set_path_effects([path_effects.Stroke(linewidth=1.5, foreground='black'), path_effects.SimpleLineShadow(),path_effects.Normal()])
                HLvalue = ax.text(lon[mxy[i], mxx[i]], (lat[mxy[i], mxx[i]]-yoffset), str(int(data[mxy[i], mxx[i]])), color=color, size=10, clip_on=True, fontweight='bold', horizontalalignment='center', verticalalignment='top', transform=transform, zorder=95)                
                HLvalue.set_path_effects([path_effects.Stroke(linewidth=1, foreground='black'), path_effects.SimpleLineShadow(),path_effects.Normal()])

    #Take in lats and lons from grb file
    lats, lons = grb.latlons()

    #import matplotlib.gridspec as gridspec
    #Creating the map figure
    map_crs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
    fig = plt.figure(1, figsize=(14, 12), facecolor='black')
    #gs = gridspec.GridSpec(12, 36, figure=fig) #[ytop:ybot, xleft:xright]
    ax = plt.subplot(1, 1, 1, projection=datacrs ) #ax = plt.subplot(gs[:-1, :], projection=datacrs )
    ax.set_extent(EXTENT, ccrs.PlateCarree())
    #ax2 = plt.subplot(gs[-1, 1:11]) #top plot
    #ax3 = plt.subplot(gs[-1, 13:23]) #middle plot
    #ax4 = plt.subplot(gs[-1, 25:35]) #bottom plot

    #Add features to the map
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'ocean', '10m', facecolor='steelblue', zorder=50))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', facecolor='gainsboro', zorder=52))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'lakes', '10m', facecolor='steelblue', linewidth=0.5, zorder=60))    
    ax.add_feature(cfeature.NaturalEarthFeature('cultural', 'admin_0_countries', '10m', facecolor='none', edgecolor='black', linewidth=1, zorder=70))
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'coastline', '10m', facecolor='none', edgecolor='black', linewidth=0.5, zorder=70))
    if SECTOR == 'CONUS' or SECTOR == 'North America':
        ax.add_feature(cfeature.STATES, edgecolor='black', linewidth=0.5, zorder=70)
    else:
        pass

    Clevs_Winds250 = [40,50,60,70,80,90,100,110]
    cmap_Winds250 = colors.ListedColormap(['#99E3FB','#47B6FB','#0F77F7','#AC97F5','#A267F4','#9126F5','#E118F3','#E118F3'])
    Winds250 = ax.contourf(lons, lats, sped_250mb, Clevs_Winds250, cmap=cmap_Winds250, extend="max", transform=ccrs.PlateCarree(), zorder=65)
    #cbar_Winds250 = plt.colorbar(Winds250,cax=ax3,pad=0.01,aspect=200,ticks=[75, 100, 125, 150, 175, 225, 250], orientation='horizontal')

    Clevs_Vort500 = np.arange(10,51,5)
    Vorticty = ax.contourf(lons, lats, VORT_500mb, Clevs_Vort500, cmap=cmo.tools.lighten(plt.cm.autumn_r, 0.3), extend="max", transform=ccrs.PlateCarree(), zorder=66)
    Vorticty2 = ax.contour(lons, lats, VORT_500mb, Clevs_Vort500, linewidths=0.35, colors="gray", transform=ccrs.PlateCarree(), zorder=66)
    #cbar_Vorticty = plt.colorbar(Vorticty,cax=ax2,pad=0.01,aspect=200,ticks=[10, 20, 30, 40, 50], orientation='horizontal')

    Clevs_PWAT = np.arange(0.8,2.6,0.1)
    cmap_PWAT = colors.ListedColormap(['#c0ffc0','#aefaae','#9cf39b','#89ed88','#77e776','#6bdf6a','#62d761','#58ce57','#4ec54e','#46bb46','#3fae3f','#39a038','#329331','#2c862b','#297828','#256a24','#225c21','#1f4e1d'])
    Pwats = ax.contourf(lons, lats, PWAT, Clevs_PWAT, cmap=cmap_PWAT, extend="max", transform=ccrs.PlateCarree(), zorder=64)
    #cbar_Pwats = plt.colorbar(Pwats,cax=ax4,pad=0.05,shrink=200,ticks=[0.8, 1.0, 1.5, 2.0, 2.5], orientation='horizontal')

    Clevs_GeoHeight500 = np.arange(0, 800, 6)
    GeoHeight500 = ax.contour(lons, lats, HGT_500mb, Clevs_GeoHeight500, linewidths=1.5, colors="black", transform=ccrs.PlateCarree(), zorder=68)
    ax.clabel(GeoHeight500, inline=1, fontsize=9, fmt='%d')

    Clevs_MSLP = np.arange(860, 1080, 4)
    MSLPcontour = ax.contour(lons, lats, MSLP, Clevs_MSLP, linewidths=0.65, colors="red", transform=ccrs.PlateCarree(), zorder=68)
    ax.clabel(MSLPcontour, inline=1, fontsize=9, fmt='%d')
    plot_maxmin_points(lons, lats, MSLP, 'max', 20, symbol='H', color='b',  transform=ccrs.PlateCarree())
    plot_maxmin_points(lons, lats, MSLP, 'min', 20, symbol='L', color='r', transform=ccrs.PlateCarree())

    quiver_slice = (slice(None, None, 4), slice(None, None, 4)) 
    quivers = ax.quiver(lons[quiver_slice], lats[quiver_slice], UWND_850mb[quiver_slice].m, VWND_850mb[quiver_slice].m, scale=820, transform=ccrs.PlateCarree(), alpha=0.5, zorder=67)

    Patch_PWAT = mpatches.Patch(color='limegreen', label='PWAT (>0.8 in)')
    Patch_Vort = mpatches.Patch(color='gold', label='500mb Vorticity')
    Patch_Wind = mpatches.Patch(color='#47B6FB', label='250mb Wind (>40 m/s)')
    Patch_GeoPotential = Line2D([0], [0], color='k', lw=2.5, label='500mb Heights (dam)')
    Patch_MSLP = Line2D([0], [0], color='r', lw=1, label='MSLP (hPa)')
    Patch_Wind850 = Line2D([0], [0], color='k', lw=0.1, marker=r'$\rightarrow$', ms=15, label='850mb Winds (m/s)') 

    leg = plt.legend(handles=[Patch_PWAT, Patch_Wind, Patch_Vort, Patch_GeoPotential, Patch_Wind850, Patch_MSLP], loc=3, framealpha=1, fontsize="9")
    leg.set_zorder(100)

    plt.title(modelName+' Synoptic Composite Map', loc='left', fontweight="bold", fontsize= 11, color='white')
    plt.title(validT, loc='right', fontsize= 10, color='white')

    UpperRightTitle = AnchoredText(UpperRight, loc=1, prop=dict(color='white', fontweight="bold", fontsize=8, horizontalalignment='right'), zorder=100)
    UpperRightTitle.patch.set_fc('black')
    UpperRightTitle.patch.set_alpha(0.7)
    ax.add_artist(UpperRightTitle)

    BottomRightTitle = AnchoredText("Orginal Map Concept By Tomer Burg", loc=4, prop=dict(color='white', fontweight="bold", fontsize=8, horizontalalignment='right'), zorder=100)
    BottomRightTitle.patch.set_fc('black')
    BottomRightTitle.patch.set_alpha(0.7)
    ax.add_artist(BottomRightTitle)
    
    #Save figure to your computer (save loction outlined near date selection)
    plt.savefig(fileNameSynopticComposite, dpi=200)

    #Clear figure variables for next figure creation
    plt.clf()

In [ ]:
#Handling map creation with if-then statements
print(Fore.BLUE + 'Map Creation Sequence Initiated')
print(Fore.BLACK + '--------------------------------------------')
time.sleep(1)
if jet250 == 'Y':
    JetMap250()
    print(Fore.GREEN + '250mb Jet Map Successfully Created')
    print(Fore.BLACK + '--------------------------------------------')
    time.sleep(1)
else:
    print(Fore.RED + '250mb Jet Map **NOT** Created')
    print(Fore.BLACK + '--------------------------------------------')
    time.sleep(1)
    
if vorticity500 == 'Y':
    VorticityMap500()
    print(Fore.GREEN + '500mb Vorticity Successfully Map Created')
    print(Fore.BLACK + '--------------------------------------------')
    time.sleep(1)
else:
    print(Fore.RED + '500mb Vorticity Map **NOT** Created')
    print(Fore.BLACK + '--------------------------------------------')
    time.sleep(1)
    
if thetaE850 == 'Y':
    ThetaE850()
    print(Fore.GREEN + '850mb Theta-E Map Successfully Created')
    print(Fore.BLACK + '--------------------------------------------')
    time.sleep(1)
else:
    print(Fore.RED + '850mb Theta-E Map **NOT** Created')
    print(Fore.BLACK + '--------------------------------------------')
    time.sleep(1)

if winds850 == 'Y':
    LLJMap850()
    print(Fore.GREEN + '850mb Winds Map Successfully Created')
    print(Fore.BLACK + '--------------------------------------------')
    time.sleep(1)
else:
    print(Fore.RED + '850mb Winds Map **NOT** Created')
    print(Fore.BLACK + '--------------------------------------------')
    time.sleep(1)

if frontogenesis925 == 'Y':
    FrontogenesisMap925()
    print(Fore.GREEN + '925mb Frontogenesis Map Successfully Created')
    print(Fore.BLACK + '--------------------------------------------')
    time.sleep(1)
else:
    print(Fore.RED + '925mb Frontogenesis Map **NOT** Created')
    print(Fore.BLACK + '--------------------------------------------')
    time.sleep(1)

if PWAT == 'Y':
    PWATmap()
    print(Fore.GREEN + 'PWAT Map Successfully Created')
    print(Fore.BLACK + '--------------------------------------------')
    time.sleep(1)
else:
    print(Fore.RED + 'PWAT Map **NOT** Created')
    print(Fore.BLACK + '--------------------------------------------') 
    time.sleep(1)  
    
if SWEAT == 'Y':
    SWEATmap()
    print(Fore.GREEN + 'SWEAT Map Successfully Created')
    print(Fore.BLACK + '--------------------------------------------')
    time.sleep(1)
else:
    print(Fore.RED + 'SWEAT Map **NOT** Created')
    print(Fore.BLACK + '--------------------------------------------') 
    time.sleep(1)    
    
if surfaceTemp == 'Y':
    SurfaceTempMap()
    print(Fore.GREEN + 'Surface Temp Map Successfully Created')
    print(Fore.BLACK + '--------------------------------------------')
    time.sleep(1)
else:
    print(Fore.RED + 'Surface Temp Map **NOT** Created')
    print(Fore.BLACK + '--------------------------------------------')
    time.sleep(1)
    
if apparentTemp == 'Y':
    ApparentTempMap()
    print(Fore.GREEN + 'Apparent Temp Map Successfully Created')
    print(Fore.BLACK + '--------------------------------------------')
    time.sleep(1)
else:
    print(Fore.RED + 'Apparent Temp Map **NOT** Created')
    print(Fore.BLACK + '--------------------------------------------')
    time.sleep(1)    
    
if SynopticComposite == 'Y':
    SynopticCompositeMap()
    print(Fore.GREEN + 'Synoptic Composite Map Successfully Created')
    print(Fore.BLACK + '--------------------------------------------')
    time.sleep(1)
else:
    print(Fore.RED + 'Synoptic Composite Map **NOT** Created')
    print(Fore.BLACK + '--------------------------------------------')
    time.sleep(1)